In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-01-22 09:19:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  3.80MB/s    in 0.3s    

2023-01-22 09:19:03 (3.80 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [2]:
import torch

In [3]:
f = open("input.txt", "r")

In [4]:
shakespear = f.read()

In [5]:
len(shakespear)

1115394

In [6]:
shakespear[:50]

'First Citizen:\nBefore we proceed any further, hear'

In [7]:
alphabet = sorted(list(set(shakespear)))

In [8]:
print(''.join(alphabet))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
print(len(alphabet))

65


In [10]:
tok2char = {tok: char for tok, char in enumerate(alphabet)}
char2tok = {char: tok for tok, char in enumerate(alphabet)}

In [11]:
def encode(inp: str, map_ = char2tok):
    return [map_[char] for char in inp]

def decode(inp: str, map_ = tok2char):
    return [map_[char] for char in inp]

In [12]:
shakespear_tok = torch.tensor(encode(shakespear))

In [15]:
shakespear_tok

tensor([18, 47, 56,  ..., 45,  8,  0])

In [16]:
split = 0.9
split_idx = int(split * len(shakespear_tok))
train_data = shakespear_tok[:split_idx]
val_data = shakespear_tok[split_idx:]

In [17]:
len(train_data)

1003854

In [18]:
len(val_data)

111540

In [24]:
block_size = 8 # what is the maximum context_length for predictions
batch_size = 4 # how many independent sequences in parallel

In [82]:
torch.manual_seed(1337)

In [53]:
def get_batch(data):
    starting_idxs = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[idx: idx + block_size] for idx in starting_idxs])
    y = torch.stack([data[idx+1: idx+1 + block_size] for idx in starting_idxs])
    return x, y

In [54]:
get_batch(train_data)

(tensor([[57, 43, 60, 43, 52,  1, 63, 43],
         [60, 43, 42,  8,  0, 25, 63,  1],
         [56, 42,  5, 57,  1, 57, 39, 49],
         [43, 57, 58, 63,  6,  1, 58, 46]]),
 tensor([[43, 60, 43, 52,  1, 63, 43, 39],
         [43, 42,  8,  0, 25, 63,  1, 45],
         [42,  5, 57,  1, 57, 39, 49, 43],
         [57, 58, 63,  6,  1, 58, 46, 47]]))

In [88]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time (len sequence), channels (embeddings)
x = torch.randn(B,T,C)